In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
import csv, re, os, statistics, filecmp, unidecode
import Connexion_Transfert as ct
import Import_trafics as it
import Outils
from Decorateurs import concat_df
from collections import Counter
from shapely.geometry import Point,LineString, MultiLineString
from shapely.ops import transform
from geoalchemy2 import Geometry, WKTElement
from Base_BdTopo import Import_outils as io

from Base_BdTopo import Rond_points as rp
from Base_BdTopo import Regroupement_correspondance as rc
from sqlalchemy.schema import MetaData
from sqlalchemy import inspect
from statistics import mean
from itertools import combinations

from sklearn.cluster import DBSCAN
import pyproj
from shapely.ops import transform

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 50)

In [33]:
#initiliser les données
cd23=it.Comptage_cd23(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD23\2019-CD23_trafics.xls',2019)
df_propre=cd23.ouvrirMiseEnForme()
cd23.classer_comptage_update_insert('gti_otv_pg11', 'na_2010_2019_p')
cd23.update_bdd_23('gti_otv_pg11','comptage', 'na_2010_2019_p')
cd23.donneesMens()
cd23.insert_bdd('gti_otv_pg11','comptage', 'na_2010_2019_mensuel',cd23.df_attr_mensuel)

In [51]:
list_id_comptag=[val for val in cd23.df_attr.id_comptag.tolist() for _ in (0, 1)]
donnees_type=['tmja','pc_pl']*len(cd23.df_attr.id_comptag.tolist())
annee_df=[str(cd23.annee)]*2*len(cd23.df_attr.id_comptag.tolist())
janv, fev, mars,avril,mai,juin,juil,aout,sept,octo,nov,dec=[],[],[],[],[],[],[],[],[],[],[],[]
for i in range(len(cd23.df_attr.id_comptag.tolist())) :
    for j in (janv, fev, mars) :
        j.extend([cd23.df_attr.trim1_TV.tolist()[i],cd23.df_attr.trim1_pcpl.tolist()[i]])
    for k in (avril,mai,juin) :
        k.extend([cd23.df_attr.trim2_TV.tolist()[i],cd23.df_attr.trim2_pcpl.tolist()[i]])
    for l in (juil,aout,sept) :
        l.extend([cd23.df_attr.trim3_TV.tolist()[i],cd23.df_attr.trim3_pcpl.tolist()[i]])
    for m in (octo,nov,dec) :
        m.extend([cd23.df_attr.trim4_TV.tolist()[i],cd23.df_attr.trim4_pcpl.tolist()[i]])
donnees_mens=pd.DataFrame({'id_comptag':list_id_comptag,'donnees_type':donnees_type,'annee':annee_df,'janv':janv,'fevr':fev,'mars':mars,'avri':avril,
              'mai':mai,'juin':juin,'juil':juil,'aout':aout,'sept':sept,'octo':octo,'nove':nov,'dece':dec})

In [54]:
cd23.donneesMens()